# Fine-tune MobileNet on sign lanuage Digits

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model 
import itertools 
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline


In [2]:
# check GPUs available
physical_device = tf.config.experimental.list_physical_devices('GPU')
print('num of gpus', len(physical_device))
tf.config.experimental.set_memory_growth(physical_device[0], True)

num of gpus 0


IndexError: list index out of range

### Image preparation

In [11]:
# organize data train, valid, test dirs. 
os.chdir('C:/Users/MOKIJ/Desktop/Sign-Language-Digits-Dataset')
if os.path.isdir('C:/Users/MOKIJ/Desktop/Sign-Language-Digits-Dataset/train/0')is False:
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')
    
    for i in range(0, 10):
        shutil.move(f'{i}', 'train')
        os.mkdir(f'valid/{i}')
        os.mkdir(f'test/{i}')
        
        valid_sample = random.sample(os.listdir(f'train/{i}'), 30)
        for j in valid_sample:
            shutil.move(f'train/{i}/{j}', f'valid/{i}')
        
        test_samples = random.sample(os.listdir(f'train/{i}'), 5)
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}')
        

In [12]:
# get the data from folders for train , valid, test
train_path = 'C:/Users/MOKIJ/Desktop/Sign-Language-Digits-Dataset/train'
valid_path = 'C:/Users/MOKIJ/Desktop/Sign-Language-Digits-Dataset/valid'
test_path = 'C:/Users/MOKIJ/Desktop/Sign-Language-Digits-Dataset/test'

In [13]:
# to make the data into format code be understood when using fit func. using keras generator
# this function is used for preprocessing the images before they get passed to the network -- (vgg16 is popular model)
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory= train_path, target_size=(224,224), batch_size=10)
# target size this is the hight and width that we want the cats and dogs being resized to if u want to scale them up or scale them down (to specify them before pass them to the network)  
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory= valid_path, target_size=(224,224), batch_size=10)
# calsses is potential labels for the data set 
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory= test_path, target_size=(224,224), batch_size=10 ,shuffle=False)


Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [14]:
assert train_batches.n == 1712 
assert valid_batches.n == 300 
assert test_batches.n == 50 


# Modify Model

In [15]:
MO = tf.keras.applications.mobilenet.MobileNet()

17227776/17225924 [==============================] - 12s 1us/step


In [17]:
MO.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [21]:
params = count_params(MO)

NameError: name 'count_params' is not defined

In [22]:
x = MO.layers[-6].output
output = Dense(units=10, activation='softmax')(x)

In [25]:
model = Model(inputs = MO.input, outputs = output)

In [26]:
for layer in model.layers[:-23]:
    layer.trainable = False 

In [28]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [29]:
params = count_params(model)
assert params['Non-trainable params']== 1365184
assert params['Trainable params']== 1873930

NameError: name 'count_params' is not defined

# train the model

In [33]:
model.compile(optimizer= Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
# run for more than 30 epochs to see better results 
model.fit(x=train_batches, validation_data=valid_batches, epochs=30, verbose=2)

Train for 172 steps, validate for 30 steps
Epoch 1/30
172/172 - 248s - loss: 0.8092 - accuracy: 0.7558 - val_loss: 1.1116 - val_accuracy: 0.6067
Epoch 2/30
172/172 - 241s - loss: 0.1290 - accuracy: 0.9854 - val_loss: 0.9501 - val_accuracy: 0.6667
Epoch 3/30
172/172 - 234s - loss: 0.0459 - accuracy: 0.9988 - val_loss: 0.7552 - val_accuracy: 0.7567
Epoch 4/30
172/172 - 238s - loss: 0.0239 - accuracy: 1.0000 - val_loss: 0.6645 - val_accuracy: 0.7967
Epoch 5/30
172/172 - 245s - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.6466 - val_accuracy: 0.7967
Epoch 6/30
172/172 - 265s - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.6148 - val_accuracy: 0.8100
Epoch 7/30
172/172 - 225s - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.5729 - val_accuracy: 0.8200
Epoch 8/30
172/172 - 228s - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.5580 - val_accuracy: 0.8167
Epoch 9/30
172/172 - 240s - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.5318 - val_accuracy: 0.8300
Epoch 10/30
172/172 - 227s - loss: 0.00

# Predict sign languages digits 

In [40]:
test_labels = test_batches.classes

In [41]:
predictions = model.predict(x=test_batches, verbose=0)

In [42]:
cm = confusion_matrix(y_true=test_labels, y_pred= predictions.argmax(axis=1))

In [43]:
test_batches.class_indices

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [44]:
 def plot_confusion_matrix(cm, 
                           classes, 
                           normalize=False, 
                           title='Confusion matrix', 
                           cmap= plt.cm.Blues):
    
   # this function prints and plots the confusion matrix.
   # normalization can be applid by setting 'normalize= True'.
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
      cm = cm.astype('float')/ cm.sum(axis=1)[:, np.newaxis]
      print('normalized confusion matrix')
    
    else:
        print('confusion matrix, without normalization')
    
    
    print(cm)
    
    thresh = cm.max()/2
    
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j,i,cm[i, j],
                horizontalalignment='center',
                color="black" if cm[i,j] > thresh else 'black') 

      plt.tight_layout()
      plt.ylabel('True label')
      plt.xlabel('Predicted label')


In [45]:
# train the model for more epochs to see better results
cm_plot_labels = ['0','1','2','3','4','5','6','7','8','9']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

confusion matrix, without normalization
[[5 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 1 0 0]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 5 0 0 0]
 [0 0 1 0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0 3 1 1]
 [0 0 0 0 0 0 0 1 0 4]]


RuntimeError: In FT2Font: Can not load face (error code 0x55)

RuntimeError: In FT2Font: Can not load face (error code 0x55)

<Figure size 432x288 with 2 Axes>